In [16]:
import os
import sys
sys.path.append(os.getcwd() + '/venv/lib/python3.6/site-packages')
sys.path

['',
 '/home/saul/anaconda3/lib/python36.zip',
 '/home/saul/anaconda3/lib/python3.6',
 '/home/saul/anaconda3/lib/python3.6/lib-dynload',
 '/home/saul/anaconda3/lib/python3.6/site-packages',
 '/home/saul/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/saul/.ipython',
 '/home/saul/PycharmProjects/Neural/venv/lib/python3.6/site-packages',
 '/home/saul/PycharmProjects/Neural/venv/lib/python3.6/site-packages']

In [17]:
import numpy as np
import math
import matplotlib.pyplot as plt
import cv2


def sigmoid(x):
    """сигмоидальная функция, работает и с числами, и с векторами (поэлементно)"""
    return 1 / (1 + np.exp(round(-x, 3)))


def sigmoid_prime(x):
    """производная сигмоидальной функции, работает и с числами, и с векторами (поэлементно)"""
    return sigmoid(x) * (1 - sigmoid(x))


def threshold(x):
    if x > 0:
        return 1
    else:
        return 0


def relu(x):
    return max(x, 0)


def softplus(x):
    return math.log(1 + np.exp(round(x, 3)))


def linear(x):
    return x


def linear_derivative(x):
    return 1


def quadratic(x):
    return x * x


def quadratic_der(x):
    return 2 * x


def save(name='', fmt='png'):
    pwd = os.getcwd()
    path = '../resources/pictures/{}'.format(fmt)
    if not os.path.exists(path):
        os.mkdir(path)
    os.chdir(path)
    plt.savefig('{}.{}'.format(name, fmt), fmt=fmt)
    os.chdir(pwd)
    plt.close()


def print_graph(x, y):
    fig = plt.figure()

    plt.plot(x, y)
    plt.show()
    
    
def read_matrix_from_file(file_name):
    f = open(file_name)
    matrix = []
    while True:
        line = f.readline()[:-1]
        if len(line) == 0:
            break
        matrix_row = line.split(' ')
        matrix.append([float(v) for v in matrix_row])
    f.close()
    return np.array(matrix)


def init_random_weights_in_file(file_name, neuron_count, neuron_weights_count, range_from, range_to):
    f = open(file_name, 'a')
    for i in range(neuron_count):
        res_str = ''
        for j in range(neuron_weights_count):
            res_str += str(round(random.uniform(range_from, range_to), 4)) + ' '
        f.write(res_str[:-1] + '\n')
    f.write('\n')
    f.close()


def save_matrix_to_file(file_name, matrix, mode='w'):
    f = open(file_name, mode)
    for row in matrix:
        f.write(np.array2string(row, max_line_width=10000, formatter={'float_kind': lambda x: "%.6f" % x})[1:-1] + '\n')
    f.write('\n')
    f.close()


def read_char_images_from_dir(directory_path, colorized, x, y):
    res_arr = []
    for filename in sorted(os.listdir(directory_path)):
        f_path = directory_path + '/' + filename
        if colorized:
            res_arr.append([cv2.resize(cv2.imread(f_path, cv2.IMREAD_COLOR), (x, y))])
        else:
            res_arr.append([cv2.resize(cv2.imread(f_path, cv2.IMREAD_GRAYSCALE), (x, y))])
    return np.array(res_arr)


def read_all_char_examples_with_answers(directory_path, colorized, x=36, y=27):
    dirs = os.listdir(directory_path)
    diag = np.zeros((len(dirs), len(dirs)), int)
    np.fill_diagonal(diag, 1)
    examples = []
    answers = []
    for i, directory in enumerate(sorted(dirs)):
        one_char_exs = read_char_images_from_dir\
            (directory_path + '/' + directory, colorized, x, y)
        examples.extend(one_char_exs)
        for _ in range(len(one_char_exs)):
            answers.append(diag[i])
    return np.array(examples), np.array(answers)


def j_quadratic(y_hat, y):
    """
    Оценивает значение квадратичной целевой функции.

    y - матрица правильных ответов (n, N)
    y_hat - матрица предсказаний (n, N)
    Возвращает значение J (число)
    """

    return 0.5 * np.mean((y_hat - y) ** 2)


def j_quadratic_derivative(y_hat, y):
    """
    Вычисляет матрицу частных производных целевой функции по каждому из предсказаний.
    y - матрица правильных ответов (n, N)
    y_hat - матрица предсказаний (n, N)
    """

    return (y_hat - y) / (len(y) * y.shape[1])


def j_cross_entropy(y_hat, y):
    return -1 * np.mean(y * np.vectorize(math.log)(y_hat) + (1 - y) * np.vectorize(math.log)(1 - y_hat))


def j_cross_entropy_derivative(y_hat, y):
    return (y_hat - y) / (y_hat * (1 - y_hat) * len(y) * y.shape[1])


In [18]:
class Neuron:

    def __init__(self, weights, activation_function=sigmoid, activation_function_derivative=sigmoid_prime):
        """
        weights - вертикальный вектор весов нейрона формы (m, 1), weights[0][0] - смещение
        activation_function - активационная функция нейрона, сигмоидальная функция по умолчанию
        activation_function_derivative - производная активационной функции нейрона
        """

        assert weights.shape[1] == 1, "Incorrect weight shape"

        self.w = weights
        self.activation_function = activation_function
        self.activation_function_derivative = activation_function_derivative

    def forward_pass(self, single_input):
        """
        активационная функция логистического нейрона
        single_input - вектор входов формы (m, 1),
        первый элемент вектора single_input - единица (если вы хотите учитывать смещение)
        """

        result = 0
        for i in range(self.w.size):
            result += float(self.w[i] * single_input[i])
        return self.activation_function(result)

    def summatory(self, input_matrix):
        """
        Вычисляет результат сумматорной функции для каждого примера из input_matrix.
        input_matrix - матрица примеров размера (n, m) без смещения, каждая строка - отдельный пример,
        n - количество примеров, m - количество переменных.
        Возвращает вектор значений сумматорной функции размера (n, 1).
        """
        return input_matrix.dot(self.w)

    def activation(self, summatory_activation):
        """
        Вычисляет для каждого примера результат активационной функции,
        получив на вход вектор значений сумматорной функций
        summatory_activation - вектор размера (n, 1),
        где summatory_activation[i] - значение суммматорной функции для i-го примера.
        Возвращает вектор размера (n, 1), содержащий в i-й строке
        значение активационной функции для i-го примера.
        """
        return np.vectorize(self.activation_function)(summatory_activation)

    def derivative(self, summatory_activation):
        """
        Вычисляет для каждого примера результат производной активационной функции,
        получив на вход вектор значений сумматорной функций
        summatory_activation - вектор размера (n, 1),
        где summatory_activation[i] - значение суммматорной функции для i-го примера.
        Возвращает вектор размера (n, 1), содержащий в i-й строке
        значение производной активационной функции для i-го примера.
        """
        return np.vectorize(self.activation_function_derivative)(summatory_activation)

    def update_mini_batch(self, x, errors, learning_rate):
        """
        x - матрица размера (batch_size, m)
        errors - вектор ошибок (batch_size, 1)
        learning_rate - константа скорости обучения
        """

        grad = x.T.dot(errors)
        self.w = self.w - grad * learning_rate

In [19]:
import random


class Layer:
    def __init__(self, neurons):
        self.neurons = neurons
        self.intermediate_sums = np.array([[]])
        self.intermediate_activations = np.array([[]])

    def process_input(self, input_matrix):
        """
        :param input_matrix: Матрица входов для слоя, (n, m)
        :return: Матрица активаций на выходе слоя
        """

        sums = np.array([neuron.summatory(input_matrix).flatten()
                         for neuron in self.neurons]).T
        activations = np.array([v.activation(sums[:, i])
                                for i, v in enumerate(self.neurons)]).T
        self.intermediate_sums = sums
        self.intermediate_activations = activations
        return activations

    def get_acts_by_sums_derivative(self):
        """
        :return: Матрица производных активационных функций по суммам для каждого нейрона, (n, m)
        """
        return np.array([v.derivative(self.intermediate_sums[:, i])
                         for i, v in enumerate(self.neurons)]).T

    def update_mini_batch(self, x, errors, learning_rate):
        """
        x - матрица размера (batch_size, m)
        errors - матрица ошибок (batch_size, N), N - число нейронов в слое
        learning_rate - константа скорости обучения
        """

        for i, v in enumerate(self.neurons):
            v.update_mini_batch(x, errors[:, [i]], learning_rate)

    def get_error(self, next_layer_errors, next_layer_weights):
        """
        Считает ошибку на данном слое сети
        next_layer_errors - ndarray размера (n, n_{l+1})
        weights - ndarray размера (n_{l+1}, n_l+1)
        :return: матрица ошибок (n, n_l)
        """

        sums = self.intermediate_sums
        sum_primes = np.array([v.derivative(sums[:, i])
                               for i, v in enumerate(self.neurons)])
        return (next_layer_weights[:, 1:].T.dot(next_layer_errors.T) * sum_primes).T

    def get_weights(self):
        """
        :return: Матрица весов для всего слоя, (n_l, n_{l-1})
        """
        return np.array([v.w.flatten() for i, v in enumerate(self.neurons)])

    @staticmethod
    def init_with_weights(weights, act_func, act_func_der):
        return Layer(
            [Neuron(weights_row.reshape(weights.shape[1], 1), act_func, act_func_der) for weights_row in weights])

    @staticmethod
    def layer_with_random_weights(neuron_count, weights_len, act_func, act_func_der):
        return Layer([Neuron(np.array([[random.uniform(-1.0, 1.0)] for j in range(weights_len + 1)]),
                             act_func, act_func_der) for i in range(neuron_count)])

In [20]:
class Network:

    def __init__(self, layers, target_function, target_function_derivative):
        self.layers = layers
        self.target_function = target_function
        self.target_function_derivative = target_function_derivative

    def process_input(self, x, y):
        """
        Выполняет один проход по сети для матрицы входных данных x.

        :param x: Вектор входов, (n, m),
        n - количество примеров
        m - количество переменных(равно числу нейронов в 1-м слое сети)
        :param y: Матрица правильных ответов, (n, N),
        N - Число нейронов в выходном слое
        :return: Значение целевой функции сети
        """

        return self.target_function(self.get_result_matrix(x), y)

    def get_result_matrix(self, x):
        """
        Возвращает матрицу активаций выходного слоя
        :param x: Матрица входов, (n, m)
        :return: Матрица выходных активаций, (n, N)
        """

        next_layer_input = np.append(np.ones((x.shape[0], 1), dtype=int), x, axis=1)
        for layer in self.layers:
            activations = layer.process_input(next_layer_input)
            next_layer_input = np.append(np.ones((len(activations), 1), dtype=int), activations, axis=1)
        return next_layer_input[:, 1:]

    def sgd(self, x, y, batch_size, learning_rate, step_limit, eps=1e-6, visualize=False):
        """

        :param x: Матрица входов - (n, m), n - число примеров, m - число переменных
        :param y: Матрица правильных ответов - (n, N), n - число примеров,
        N - число нейронов выходного слоя
        :param batch_size: Размер батча, выбираемого для обучения
        :param learning_rate: Константа скорости обучения
        :param step_limit: Максимальное число шагов, которое может сделать алгоритм
        :param eps: Точность алгоритма
        :param visualize: Визуализировать ли изменение целевой функции
        :return: 1, если успешно сошлось, 0 если достигнут предел количества допустимых шагов
        """

        errors = 0
        step = 0
        steps = []
        target_func_results = []
        while (not errors) and (step < step_limit):
            batch_ids_arr = Network.get_batches(np.arange(len(y)), batch_size)
            init_target_func = self.process_input(x, y)
            for batch_ids in batch_ids_arr:
                x_b = x[batch_ids]
                y_b = y[batch_ids]
                self.update_mini_batch(x_b, y_b, learning_rate)
                step += 1
                if visualize:
                    steps.append(step)
                    target_func_results.append(self.process_input(x, y))
            res_target_func = self.process_input(x, y)
            errors += int(abs(init_target_func - res_target_func) < eps)
        if visualize:
            print_graph(steps, target_func_results)
        return errors

    def update_mini_batch(self, x, y, learning_rate):
        """
        Обновляет все веса сети для одного батча
        :param x: Матрица входов - (batch_size, m), batch_size - число примеров, m - число переменных
        :param y: Матрица правильных ответов - (batch_size, N), batch_size - число примеров,
        N - число нейронов выходного слоя
        :param learning_rate: Константа скорости обучения
        """
        result_matrix = self.get_result_matrix(x)
        last_layer = self.layers[-1]
        da_dz = last_layer.get_acts_by_sums_derivative()

        layer_error = self.target_function_derivative(result_matrix, y) * da_dz
        layer_weights = last_layer.get_weights()

        last_layer_id = len(self.layers) - 1
        reversed_layers = self.layers[::-1]
        for i, v in enumerate(reversed_layers):
            if i < last_layer_id:
                layer_input = reversed_layers[i + 1].intermediate_activations
            else:
                layer_input = x
            layer_input = np.append(np.ones((layer_input.shape[0], 1), dtype=int), layer_input, axis=1)

            v.update_mini_batch(layer_input, layer_error, learning_rate)
            if i < last_layer_id:
                layer_error = reversed_layers[i + 1].get_error(layer_error, layer_weights)
                layer_weights = reversed_layers[i + 1].get_weights()
        return layer_error

    @staticmethod
    def get_batches(x, batch_size):
        """
        Делит массив x на батчи размера n
        :param x: Массив для деления
        :param batch_size: Размер батча
        :return: Массив батчей
        """
        np.random.shuffle(x)
        return np.array([x[i:i + batch_size] for i in range(0, len(x), batch_size)])

In [21]:
import scipy.signal as signal


class ConvolutionNeuron:
    """
    Нейрон, сворачивающий карту с помощью ядра.
    kernel - ядро для свертки.
    """
    def __init__(self, kernel, subsample_size=2):

        assert kernel.shape[0] == kernel.shape[1] and kernel.shape[0] % 2 == 1, "Kernel must be squared and odd-sized!"

        self.kernel = kernel
        self.subsample_size = subsample_size
        self.convolution_res = []

    def process_sign_maps(self, sign_maps):
        """
        Сворачивает входной массив карт признаков ядром и возвращает новый массив.
        :param sign_maps: массив карт для свертки, одна карта разммера (n, m)
        :return: массив новых карт, полученный в процессе свертки,
         одна карта размера (n - kernel.x + 1, m - kernel.y + 1)
        """
        self.convolution_res = []
        return np.array([self.process_sign_map(sign_map) for sign_map in sign_maps])

    def process_sign_map(self, sign_map):
        """
        Сворачивает входную карту признаков ядром и возвращает новую карту.
        :param sign_map: карта для свертки, (n, m)
        :return: новая карта, полученный в процессе свертки, (n - kernel.x + 1, m - kernel.y + 1)
        """
        res_map = signal.convolve2d(sign_map, self.kernel, 'valid')
        self.convolution_res.append(res_map)
        return self.subsample_one_map(res_map)

    def update_mini_batch(self, neuron_input, error, learning_rate):
        grad = np.zeros_like(self.kernel)
        for i in range(neuron_input.shape[0]):
            grad += self.get_grad_for_one_example(neuron_input[i], error[i])
        self.kernel -= grad * learning_rate

    def get_grad_for_one_example(self, neuron_input, error):
        grad = np.zeros_like(self.kernel)
        kernel_y = self.kernel.shape[0]
        kernel_x = self.kernel.shape[1]
        for y in range(error.shape[0]):
            for x in range(error.shape[1]):
                grad += neuron_input[y:y + kernel_y, x:x + kernel_x] * error[y, x]
        return grad / (error.shape[0] * error.shape[1])

    def subsample_one_map(self, sign_map):
        ssize = self.subsample_size
        output = np.zeros((int(math.ceil(sign_map.shape[0] / ssize)),
                           int(math.ceil(sign_map.shape[1] / ssize))))
        for y in range(output.shape[0]):
            for x in range(output.shape[1]):
                output[y, x] = np.amax(sign_map[y * ssize:y * ssize + ssize,
                                       x * ssize:x * ssize + ssize])
        return output

    def create_err_map(self, next_layer_err, next_layer_weights, ex_id):
        err_on_subs = self.create_err_map_for_subs(next_layer_err, next_layer_weights)
        err_on_convs = self.create_err_map_for_conv(err_on_subs, ex_id)
        return err_on_convs

    @staticmethod
    def create_err_map_for_subs(next_layer_err, next_layer_weights):
        reversed_kernel = np.fliplr(np.flipud(next_layer_weights))
        return signal.convolve2d(next_layer_err, reversed_kernel, 'full')

    def create_err_map_for_conv(self, sub_layer_err, ex_id):
        input_map = np.array(self.convolution_res)[ex_id]
        max_values_primes_iter = iter(sub_layer_err.flatten())
        output = np.zeros_like(input_map)
        for y in range(sub_layer_err.shape[0]):
            for x in range(sub_layer_err.shape[1]):
                start_y = y * self.subsample_size
                start_x = x * self.subsample_size
                wind = input_map[start_y:start_y + self.subsample_size, start_x:start_x + self.subsample_size]
                (a, b) = np.unravel_index(np.argmax(wind, axis=None), wind.shape)
                oid_y = min(y * self.subsample_size + a, input_map.shape[0] - 1)
                oid_x = min(x * self.subsample_size + b, input_map.shape[1] - 1)
                output[oid_y, oid_x] = next(max_values_primes_iter)
        return output

In [22]:
class ConvolutionLayer:
    """
    Класс, представляющий слой сверточной сети(сверки или подвыборки в зависимости от нейронов)
    neurons = ядра свертки слоя
    """
    def __init__(self, neurons):
        self.neurons = neurons
        self.saved_maps = []

    def process_sign_maps(self, sign_maps_arr):
        """
        Применяет каждый нейрон слоя к входному массиву карт признаков

        :param sign_maps_arr: Входной массив карт признаков для каждого примера
        :return: Массив с новыми картами признаков для каждого примера
        """
        self.clear_convolution_results()
        self.saved_maps = np.array([self.process_maps_one_example(maps) for maps in sign_maps_arr])
        return self.saved_maps

    def process_maps_one_example(self, maps):
        res = []
        neurons_for_map = int(len(self.neurons) / len(maps))
        for i in range(len(maps)):
            start = i * neurons_for_map
            for neuron in self.neurons[start:start + neurons_for_map]:
                res.append(neuron.process_sign_map(maps[i]))
        return np.array(res)

    def get_errors(self, next_layer_errs, next_layer_weights):
        res = []
        for ex_id in range(len(next_layer_errs)):
            res.append(self.create_error_maps(next_layer_errs[ex_id], next_layer_weights, ex_id))
        return np.array(res)

    def create_error_maps(self, next_layer_errs, next_layer_weights, ex_id):
        return np.array([neuron.create_err_map(next_layer_errs[i], next_layer_weights[i], ex_id)
                        for i, neuron in enumerate(self.neurons)])

    def update_mini_batch(self, layer_input, errors, learning_rate):
        input_id = 0
        neurons_for_map = int(len(self.neurons) / layer_input.shape[1])

        for i, v in enumerate(self.neurons):
            v.update_mini_batch(layer_input[:, input_id], errors[:, i], learning_rate)
            if i == input_id * neurons_for_map + neurons_for_map:
                input_id += 1

    def get_weights(self):
        return np.array([neuron.kernel for neuron in self.neurons])

    def clear_convolution_results(self):
        for neuron in self.neurons:
            neuron.convolution_res = []

    @staticmethod
    def init_random_convolution_layer(neuron_count, kernel_size, subsampling_size=2):
        return ConvolutionLayer(np.array([ConvolutionNeuron(np.random.rand(kernel_size, kernel_size) - 0.5,
                                                            subsampling_size) for i in range(neuron_count)]))

    @staticmethod
    def init_with_weights_from_file(file_name, kernel_sie, subsampling_size):
        matrix = read_matrix_from_file(file_name)
        return ConvolutionLayer(np.array([ConvolutionNeuron(v.reshape(kernel_sie, kernel_sie), subsampling_size)
                                for i, v in enumerate(matrix)]))

In [23]:
class ConvolutionNetwork:
    def __init__(self, layers, fully_connected_net):
        self.layers = layers
        self.fully_connected_net = fully_connected_net

    def process_input(self, input_data, y):
        next_layer_input = input_data
        for layer in self.layers:
            next_layer_input = layer.process_sign_maps(next_layer_input)

        # next_layer_input = np.maximum(next_layer_input, 0)
        return self.fully_connected_net.process_input(self.transform_maps(next_layer_input), y)

    def sgd(self, x, y, batch_size, learning_rate, learning_rate_conn, step_limit, eps=1e-6, visualize=False):
        errors = 0
        step = 0
        steps = []
        target_func_results = []
        while (not errors) and (step < step_limit):
            batch_ids_arr = self.fully_connected_net.get_batches(np.arange(len(y)), batch_size)
            init_target_func = self.process_input(x, y)
            for batch_ids in batch_ids_arr:
                x_b = x[batch_ids]
                y_b = y[batch_ids]
                self.update_mini_batch(x_b, y_b, learning_rate, learning_rate_conn)
                step += 1
                if visualize:
                    steps.append(step)
                    target_func_results.append(self.process_input(x, y))
            res_target_func = self.process_input(x, y)
            errors += int(abs(init_target_func - res_target_func) < eps)
        if visualize:
            print_graph(steps, target_func_results)
        return errors

    def update_mini_batch(self, x, y, learning_rate, learning_rate_conn):
        self.process_input(x, y)
        transformed = self.transform_maps(self.layers[-1].saved_maps)
        errors = self.fully_connected_net.update_mini_batch(transformed, y, learning_rate_conn)
        last_conv_layer_err = self.get_last_convolution_layer_err\
            (errors, self.fully_connected_net.layers[0].get_weights())

        layer_err = last_conv_layer_err
        last_layer_id = len(self.layers) - 1
        reversed_layers = self.layers[::-1]
        layer_weights = self.layers[-1].get_weights()

        for i, v in enumerate(reversed_layers):
            if i < last_layer_id:
                layer_input = reversed_layers[i + 1].saved_maps
            else:
                layer_input = x
            v.update_mini_batch(layer_input, layer_err, learning_rate)
            if i < last_layer_id:
                layer_err = reversed_layers[i + 1].get_errors(layer_err, layer_weights)
                layer_weights = reversed_layers[i + 1].get_weights()

        return layer_err

    def get_last_convolution_layer_err(self, next_layer_errors, next_layer_weights):
        last_layer = self.layers[-1]
        last_layer_map_size = last_layer.saved_maps[0].shape[1:]

        err_sub_lauer = (next_layer_weights[:, 1:].T.dot(next_layer_errors.T)).T
        err_maps_sub_layer = ConvolutionNetwork.transform_err_arrays_to_maps\
            (err_sub_lauer, last_layer_map_size[0], last_layer_map_size[1])

        final_result = []
        for ex_id in range(len(next_layer_errors)):
            ex_result = []
            for i, neuron in enumerate(last_layer.neurons):
                ex_result.append(neuron.create_err_map_for_conv(err_maps_sub_layer[ex_id][i], ex_id))
            final_result.append(np.array(ex_result))
        return np.array(final_result)

    @staticmethod
    def transform_maps(maps_arr):
        return np.array([maps.flatten() for maps in maps_arr])

    @staticmethod
    def transform_err_arrays_to_maps(err_arrays, map_y, map_x):
        return np.array([ConvolutionNetwork.transform_err_array_to_maps(err_arr, map_y, map_x)
                         for err_arr in err_arrays])

    @staticmethod
    def transform_err_array_to_maps(err_arr, map_y, map_x):
        elems_count = map_y * map_x
        return np.array([err_arr[i:i + elems_count].reshape(map_y, map_x)
                         for i in range(int(len(err_arr) / elems_count))])

In [24]:
def transform_input_imgs_to_data(imgs):
    data = np.array(list((map(lambda arr: arr.flatten(), imgs)))) / 255
    return abs(1 - data)


def run_conv_net():
    conv_layer1 = ConvolutionLayer.init_with_weights_from_file('resources/weights_kernels0', 5, 2)
    conv_layer2 = ConvolutionLayer.init_with_weights_from_file('resources/weights_kernels1', 5, 2)
    conv_layer3 = ConvolutionLayer.init_with_weights_from_file('resources/weights_kernels2', 3, 2)
    # conv_layer1 = ConvolutionLayer.init_random_convolution_layer(8, 5, 2)
    # conv_layer2 = ConvolutionLayer.init_random_convolution_layer(16, 5, 2)
    # conv_layer3 = ConvolutionLayer.init_random_convolution_layer(32, 3, 2)
    layer1 = Layer.layer_with_random_weights(100, 768, sigmoid, sigmoid_prime)
    layer2 = Layer.layer_with_random_weights(10, 100, sigmoid, sigmoid_prime)

    # network_conv = Network([layer1, layer2], j_cross_entropy, j_cross_entropy_derivative)
    network_conv = Network(
        [Layer.init_with_weights(read_matrix_from_file('resources/weights_conv' + str(i)), sigmoid, sigmoid_prime)
         for i in range(2)], j_cross_entropy, j_cross_entropy_derivative)

    network = ConvolutionNetwork([conv_layer1, conv_layer2, conv_layer3], network_conv)
    exs, ans = read_all_char_examples_with_answers('resources/digits', False, 64, 48)
    exs = abs(1 - exs / 255)

    print(network.sgd(exs, ans, 110, 0.1, 0.1, 1, 1e-9, visualize=True))

    for i, l in enumerate(network.fully_connected_net.layers):
        save_matrix_to_file('resources/weights_conv' + str(i), l.get_weights())

    for i, l in enumerate(network.layers):
        save_matrix_to_file('resources/weights_kernels' + str(i), map(lambda x: x.flatten(), l.get_weights()))

    print(network.process_input(exs, ans))


def run_simple_net():
    network = Network(
        [Layer.init_with_weights(read_matrix_from_file('resources/weights' + str(i)), sigmoid, sigmoid_prime) for i in
         range(2)],
        j_cross_entropy,
        j_cross_entropy_derivative)

    # data = transform_input_imgs_to_data(read_char_images_from_dir('resources/test', False, 36, 27))
    data, answers = read_all_char_examples_with_answers('resources/digits', False)

    data = transform_input_imgs_to_data(data)
    print(network.sgd(data, answers, 110, 10, 10, eps=1e-7, visualize=True))

    for i, l in enumerate(network.layers):
        save_matrix_to_file('resources/weights' + str(i), l.get_weights())

    print(network.get_result_matrix(data))

    print(network.process_input(data, answers))

run_conv_net()
# run_simple_net()
sys.exit(0)


KeyboardInterrupt: 